In [1]:
# First create the kh2lib object
from kh2lib.kh2lib import kh2lib
import os, json
lib = kh2lib()
if not os.path.isdir("workspace"):
    os.mkdir("workspace")

In [17]:
# Now we extract the xemnas BAR into a folder in our workspace
MDLX = "tt01.ard"
BAR = os.path.join(lib.gitpath, "KH2","ard",MDLX)
assert os.path.isfile(BAR)
lib.editengine.bar_extract(BAR, os.path.join(os.getcwd(), "workspace","tt01"))

In [9]:
MDLX = "tt01.ard"
BAR = os.path.join(lib.gitpath, "KH2","ard",MDLX)
lib.editengine.bar_build(os.path.join(os.getcwd(), "workspace", "tt01"), BAR)

In [ ]:
# Open the workspace in a window and copy the b_ex.ai to have an unmodified copy to diff against
# Then open the b_ex.ai in a hex editor and make the following changes to the bottom of the file
# Find the header definition for these two functions, and replace them with 00s
#  warp_building_front
#  rc_invitation_to_dark
# Alternatively, just edit a decompiled version of this AI file in Ghidra (using Govanify's KH2 plugin)

base_file = os.path.join("workspace", "battle", "enmp.list.base")
mod_file = os.path.join("workspace", "riku", "n_ex.ai")
assert os.path.isfile(base_file)
assert os.path.isfile(mod_file)

# go to the roxas fight, and create a memdump using emuhaste
#  it is recommended to make a savestate in the cutscene before the fight loads, for ease of testing
# Place the memdump in the workspace folder, using whatever name you choose
dumpname = "realriku.dump"
subdata = lib.parsingengine.load_memdump(os.path.join("workspace", dumpname))
# SUBDATA = lib.parsingengine.load_memdump(base_file) #IF THINGS ARENT WORKING LOOK HERE SILLY

starting_offset = int(lib.parsingengine.locate_file(base_file),16)
print(hex(starting_offset))
0/0
lib.cheatengine.pnach = []
# This code will diff the two versions of the ai file and create codes to make those same changes via RAM
codes = lib.parsingengine.generate_codes(str(starting_offset), base_file, mod_file)
for code in codes:
    print(code)
print(codes)
# The memdump skips the first section of memory addresses so the code offsets need to be fixed
codes = [code[1].replace("0","2") + code[1:] for code in codes]
codes
# codes = []#DEBUG

# Now we add the codes into the pnach portion of kh2lib, and generate a pnach file in your pcsx 2 directory
# Then just load your savestate, skip cutscene and start the fight
lib.cheatengine.apply_ram_code(codes)
lib.cheatengine.write_pnach(debug=True)

In [ ]:
import time, os
while 1:
    time.sleep(0.1)
    if os.path.isfile(mod_file):
        lib.cheatengine.pnach = []
        codes = lib.parsingengine.generate_codes(str(starting_offset), base_file, mod_file)
        codes = [code[1].replace("1","2") + code[1:] for code in codes]
        lib.cheatengine.apply_ram_code(codes)
        lib.cheatengine.write_pnach(debug=False)
        print(time.time())
        os.remove(mod_file)

In [ ]:
# Spawnscript reader
opcodes = {
    0x0: ("Spawn", "Load Spawn File"),
    0x1: ("MapOcclusion", "Enable/Disable meshes/collisions"),
    0x2: ("RandomSpawn", "Randomly spawn enemies"),
    0x3: ("CasualSpawn", "Casually spawn enemy group"),
    0x4: ("Capacity", "Set capacity of map (float)"),
    0x5: ("AllocEnemy", "Memory reserved for enemies"),
    0x6: ("Unk06", "Set memory area 0034ecd8"),
    0x7: ("Unk07", "Set the memory area 0034ecdc"),
    0x9: ("UncommonSpawn", "Somewhat like Spawn, used less"),
    0xa: ("Unk0a", "No Idea"),
    0xb: ("BarrierFlag", "Defines parts of the map that are blocked by a barrier"),
    0xc: ("AreaSettings", "Play a function related to evt"),
    0xe: ("Unk0e", "Unknown"),
    0xf: ("Party", "Which party members are allowed"),
    0x10: ("BGM", "Set Background Music"),
    0x11: ("MSGWall", "Sets message played when hitting invisible wall"),
    0x12: ("Unk12", "Unknown and unused"),
    0x13: ("Camera", "Camera mode for map"),
    0x14: ("StatusFlag3", "Sets 3rd bit of memory area 0034F240"),
    0x15: ("Mission", "Loads MSN file"),
    0x16: ("Bar", "Loads BAR file by specifying path"),
    0x17: ("StatusFlag5", "Sets 5th byte of memory area 0034F240 (Tornado related?)"),
    0x18: ("AllocEffect", "Allocates memory for the particle system?"),
    0x19: ("Progress", "Set story flag"),
    0x1a: ("Visibility On", "Something with 01c60548"),
    0x1b: ("Visibility Off", "Something with 01c60550"),
    0x1c: ("If", "Recursively calls spawnscript parser somehow?"),
    0x1d: ("Unk1d", "Unknown"),
    0x1e: ("BattleLevel", "Override battle level"),
    0x1f: ("Unk1F", "Unknown but similar to CasualSpawn")
}

In [24]:
SPAWNSCRIPT_PATH = os.path.join(os.getcwd(), "workspace", "tt01")
OUTDIR = os.path.join(os.getcwd(), "workspace", "tt01", "extracted_spawnscripts")
MAP_IN_PATH = os.path.join(SPAWNSCRIPT_PATH, "map.script")
MAP_OUT_PATH = os.path.join(OUTDIR, "map.txt")
EVT_IN_PATH = os.path.join(SPAWNSCRIPT_PATH, "evt.script")
EVT_OUT_PATH = os.path.join(OUTDIR, "evt.txt")
# BTL_IN_PATH = os.path.join(SPAWNSCRIPT_PATH, "btl.script")
# BTL_OUT_PATH = os.path.join(OUTDIR, "btl.txt")

In [23]:
lib.editengine.spawnscript_extract(MAP_IN_PATH, MAP_OUT_PATH)
lib.editengine.spawnscript_extract(EVT_IN_PATH, EVT_OUT_PATH)
# lib.editengine.spawnscript_extract(BTL_IN_PATH, BTL_OUT_PATH)

In [ ]:
SPAWNSCRIPT_PATH = os.path.join(os.getcwd(), "workspace", "riku_ard")
OUTDIR = os.path.join(os.getcwd(), "workspace", "riku_ard", "extracted_spawnscripts")
MAP_IN_PATH = os.path.join(SPAWNSCRIPT_PATH, "map.script")
MAP_OUT_PATH = os.path.join(OUTDIR, "map.txt")
EVT_IN_PATH = os.path.join(SPAWNSCRIPT_PATH, "evt.script")
EVT_OUT_PATH = os.path.join(OUTDIR, "evt.txt")
BTL_IN_PATH = os.path.join(SPAWNSCRIPT_PATH, "btl.script")
BTL_OUT_PATH = os.path.join(OUTDIR, "btl.txt")

In [ ]:
lib.editengine.spawnscript_extract(MAP_IN_PATH, MAP_OUT_PATH)
lib.editengine.spawnscript_extract(EVT_IN_PATH, EVT_OUT_PATH)
lib.editengine.spawnscript_extract(BTL_IN_PATH, BTL_OUT_PATH)

In [25]:
lib.editengine.spawnscript_compile(MAP_OUT_PATH, MAP_IN_PATH)
lib.editengine.spawnscript_compile(EVT_OUT_PATH, EVT_IN_PATH)
lib.editengine.bar_build(os.path.join(os.getcwd(), "workspace", "tt01","tt01.ard.json"), BAR)
lib.patch_game(fromgit=True)

['-patchmaker', '-output', 'patch.kh2patch', '-author', 'kh2lib', '-version', '1', '-skipchangelog', '-skipcredits']
(b'KH2FM_Toolkit\nVersion PRE-3.0\nDEVELOPER RELEASE\n\nProgrammed by GovanifY\nhttp://www.govanify.com\nhttp://www.twitter.com/GovanifY\nSoftware under GPL 2 license, for more info, use the command -license\n\nThis tool is able to create patches for the software KH2FM_Toolkit.\nIt can add files using the internal compression of the game \nKingdom Hearts 2(Final Mix), relink files to others, recreate\nthe iso without size limits and without corruption.\nThis patch system is the best ever made for this game atm.\n\nPress enter to run using the file: patch.kh2patchOther information (leave blank to continue): Filenames may be formatted as text (msg/jp/lk.bar) or hash (0x030b45da).\r\n\nEnter filename:   Using "ard/es00.ard" for F5F21660\r\nRelink to this filename(ex: 000al.idx) [Blank for none]: Compress this file? [Y/n] Parent compressed file [Leave blank for KH2]:   Using

In [ ]:
os.stat(MAP_OUT_PATH)

In [ ]:
os.stat(MAP_OUT_PATH)[8]

In [ ]:
lib.editengine.spawnscript_compile(BTL_OUT_PATH, BTL_IN_PATH+".new")

In [ ]:
import time, os
map_etime = 0
btl_etime = 0
evt_etime = 0
base_file_map = os.path.join("workspace", "riku_ard", "map.script")
mod_file_map = os.path.join("workspace", "riku_ard", "map.script.new")
base_file_btl = os.path.join("workspace", "riku_ard", "btl.script")
mod_file_btl = os.path.join("workspace", "riku_ard", "btl.script.new")
base_file_evt = os.path.join("workspace", "riku_ard", "evt.script")
mod_file_evt = os.path.join("workspace", "riku_ard", "evt.script.new")
starting_offset_map = int(lib.parsingengine.locate_file(base_file_map),16)
starting_offset_btl = int(lib.parsingengine.locate_file(base_file_btl),16)
starting_offset_evt = int(lib.parsingengine.locate_file(base_file_evt),16)
num_failures = 0
last_exc = None
map_codes = []
evt_codes = []
btl_codes = []
while 1:
    if num_failures > 10:
        raise last_exc
    try:
        time.sleep(0.05)
        newCheats = False

        map_netime = os.stat(MAP_OUT_PATH)[8]
        btl_netime = os.stat(BTL_OUT_PATH)[8]
        evt_netime = os.stat(EVT_OUT_PATH)[8]
        if map_netime != map_etime:
            map_etime = map_netime
            newCheats = True
        if btl_netime != btl_etime:
            btl_etime = btl_netime
            newCheats = True
        if evt_netime != evt_etime:
            evt_etime = evt_netime
            newCheats = True
        if newCheats:
            lib.cheatengine.pnach = []
            lib.editengine.spawnscript_compile(MAP_OUT_PATH, MAP_IN_PATH+".new")
            map_codes = lib.parsingengine.generate_codes(str(starting_offset_map), base_file_map, mod_file_map)
            lib.editengine.spawnscript_compile(BTL_OUT_PATH, BTL_IN_PATH+".new")
            btl_codes = lib.parsingengine.generate_codes(str(starting_offset_btl), base_file_btl, mod_file_btl)
            lib.editengine.spawnscript_compile(EVT_OUT_PATH, EVT_IN_PATH+".new")
            evt_codes = lib.parsingengine.generate_codes(str(starting_offset_evt), base_file_evt, mod_file_evt)
            codes = [code[1].replace("1","2") + code[1:] for code in map_codes+btl_codes+evt_codes]
            lib.cheatengine.apply_ram_code(codes)
            lib.cheatengine.write_pnach(debug=False)
            print(time.time())
        num_failures = 0
    except Exception as exc:
        last_exc = exc
        print(last_exc)
        num_failures += 1